In [ ]:
"""
Option 1:

from random import choice 

nouns = ("puppy", "car", "rabbit", "girl", "monkey")
verbs = ("runs", "hits", "jumps", "drives", "barfs") 
adv = ("crazily.", "dutifully.", "foolishly.", "merrily.", "occasionally.")
adj = ("adorable", "clueless", "dirty", "odd", "stupid")

# asking user as to how many sentences he would like to generate 
for _ in range (int (input ("Enter integer value :"))): 
    print(list(map(choice, [nouns, verbs, adv, adj])))
    
"""

In [1]:
import time
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt


In [2]:
def normalize(counter):
    """ Convert a `letter -> count` counter to a list 
    of (letter, frequency) pairs, sorted in descending order of 
    frequency.

    Parameters
    -----------
    counter : collections.Counter
        letter -> count

    Returns
    -------
    List[Tuple[str, float]]
       A list of tuples: (letter, frequency) pairs in order
       of descending-frequency

    Examples
    --------
    >>> from collections import Counter
    >>> letter_count = Counter({"a": 1, "b": 3})
    >>> letter_count
    Counter({'a': 1, 'b': 3})

    >>> normalize(letter_count)
    [('b', 0.75), ('a', 0.25)]
    """

    total = sum(counter.values())
    return [(char, cnt/total) for char, cnt in counter.most_common()]


In [3]:
from collections import defaultdict

def train_lm(text, n):
    """ Train character-based n-gram language model.
        
    This will learn: given a sequence of n-1 characters, what the probability
    distribution is for the n-th character in the sequence.

    For example if we train on the text:
        text = "cacao"

    Using a n-gram size of n=3, then the following dict would be returned.
    See that we *normalize* each of the char_count_tuples for a given history

        {'ac': [('a', 1.0)],
         'ca': [('c', 0.5), ('o', 0.5)],
         '~c': [('a', 1.0)],
         '~~': [('c', 1.0)]}

    Tildas ("~") are used for padding the history when necessary, so that it's 
    possible to estimate the probability of a seeing a character when there 
    aren't (n - 1) previous characters of history available.

    So, according to this text we trained on, if you see the sequence 'ac',
    our model predicts that the next character should be 'a' 100% of the time.

    For generating the padding, recall that Python allows you to generate 
    repeated sequences easily: 
       `"p" * 4` returns `"pppp"`

    Parameters
    -----------
    text: str 
        A string (doesn't need to be lowercased).
        
    n: int
        The length of n-gram to analyze.

    Returns
    -------
    Dict[str, List[Tuple[str, float]]]
        
        {n-1 history -> [(letter, normalized count), ...]}
        
        A dictionary that maps histories (strings of length (n-1)) to lists of (char, prob) 
        pairs, where prob is the probability (i.e frequency) of char appearing after 
        that specific history.

    Examples
    --------
    >>> train_lm("cacao", 3)
    {'ac': [('a', 1.0)],
     'ca': [('c', 0.5), ('o', 0.5)],
     '~c': [('a', 1.0)],
     '~~': [('c', 1.0)]}
    """

    raw_lm = defaultdict(Counter) # history -> {char -> count}
    history = "~" * (n - 1)  # length n - 1 history
    
    # count number of times characters appear following different histories
    #
    # for char in text ...
    #    1. Increment language model's count, given current history and character
    #    2. Update history

    for char in text:
        raw_lm[history][char] += 1
        # slide history window to the right by one character
        history = history[1:] + char

    
    # create the finalized language model – a dictionary with: history -> [(char, freq), ...]
    lm = {history : normalize(counter) for history, counter in raw_lm.items()} 
    
    return lm


In [4]:
def generate_letter(lm, history):
    """ Randomly picks letter according to probability distribution associated with 
    the specified history, as stored in your language model.

    Note: returns dummy character "~" if history not found in model.

    Parameters
    ----------
    lm: Dict[str, List[Tuple[str, float]]] 
        The n-gram language model. 
        I.e. the dictionary: history -> [(char, freq), ...]

    history: str
        A string of length (n-1) to use as context/history for generating 
        the next character.

    Returns
    -------
    str
        The predicted character. '~' if history is not in language model.
    """

    if not history in lm:
        return "~"
    letters, probs = unzip(lm[history])
    i = np.random.choice(letters, p=probs)
    return i

In [5]:
def generate_text(lm, n, nletters=100):
    """ Randomly generates `nletters` of text by drawing from 
    the probability distributions stored in a n-gram language model 
    `lm`.

    Parameters
    ----------
    lm: Dict[str, List[Tuple[str, float]]]
        The n-gram language model. 
        I.e. the dictionary: history -> [(char, freq), ...]
    
    n: int
        Order of n-gram model.
    
    nletters: int
        Number of letters to randomly generate.

    Returns
    -------
    str
        Model-generated text. Should contain `nletters` number of
        generated characters. The pre-pended ~'s are not to be included. 
    """
    # <COGINST>
    history = "~" * (n - 1)
    text = []
    for i in range(nletters):
        c = generate_letter(lm, history)
        text.append(c)
        history = history[1:] + c
    return "".join(text)  

## 1. Generating "Shakespeare"

The next cell loads in Andrej Karpathy's shakespeare_input.txt file.

In [6]:
path_to_shakespeare = get_data_path("shakespeare_input.txt")

with open(path_to_shakespeare, "r") as f:
    shakespeare = f.read()
print(str(len(shakespeare)) + " character(s)")
chars = set(shakespeare)
print(f"'~' is a good pad character: {'~' not in chars}")

NameError: name 'get_data_path' is not defined

In [7]:
t0 = time.time()
lm11 = train_lm(shakespeare, 11)
t1 = time.time()
print("elapsed = " + str(t1 - t0) + "s")

NameError: name 'shakespeare' is not defined

In [8]:
print(generate_text(lm11, 11, 500))

NameError: name 'lm11' is not defined

## 2. Generating "The Percy Jackson Series"

The next cell loads in pjallbooks.txt file.

In [9]:
from cogworks_data.language import get_data_path

path_to_pj = "/Users/mohan/Desktop/cogworks/bwsi/ryan-sus/capstone/MadLib/pjallbooks.txt"

#get_data_path("pjolympians.txt")

with open(path_to_pj, "rb") as f:
    pj = f.read().decode()  
    pj = pj.lower()  
    pj.split()

lmpj1 = train_lm(pj, 50)
print(generate_text(lmpj1, 50, 1000))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mohan/Desktop/cogworks/bwsi/ryan-sus/capstone/MadLib/pjallbooks.txt'

## Working on making the actual game

1. remove random nouns using spacy
2. find the pos of the random words
3. either leave word blank for user to fill in
4. or have computer generate words

## 3. Generating "Dr. Suess' Work"

The next cell loads in drsuess.txt file.

In [30]:
import pandas as pd

path_to_seuss = "/Users/mohan/Desktop/cogworks/bwsi/ryan-sus/capstone/MadLib/drseuss.txt"

seuss = pd.read_csv(path_to_seuss, encoding= 'unicode_escape')

#with open(path_to_suess, "rb") as f:
seuss = seuss.read().decode()  
seuss = seuss.lower()
seuss.split()
    


print(str(len(suess)) + " character(s)")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 8, saw 3
